In [124]:
from preprocessing import load_data
import random
import ta

In [125]:
random.seed(420)

In [126]:
stock_data, ta_features, preview = load_data("./data/wig_d.csv")
print(ta_features.shape[1])

84


In [127]:
import inspect
import ta

def get_constructors(module):
    # Get all class names defined in the module
    class_names = [name for name in dir(module) if isinstance(getattr(module, name), type)]
    constructors = []
    # Print constructors for each class with their arguments
    for class_name in class_names:
        class_ = getattr(module, class_name)
        # Check if it's a class (not a module or function)
        if isinstance(class_, type):
            # Check if the class has a __init__ method
            if hasattr(class_, '__init__'):
                # Get the constructor signature
                init_signature = inspect.signature(getattr(class_, '__init__'))
                full_class_name = f'{module.__name__}.{class_name}'
                constructor = (class_, init_signature)
                constructors.append(constructor)
                print(constructor)
            else:
                # print(f"Class {class_name} does not have a constructor (__init__ method)")
                pass
    return constructors

constructors = get_constructors(ta.momentum)

(<class 'ta.momentum.AwesomeOscillatorIndicator'>, <Signature (self, high: pandas.core.series.Series, low: pandas.core.series.Series, window1: int = 5, window2: int = 34, fillna: bool = False)>)
(<class 'ta.utils.IndicatorMixin'>, <Signature (self, /, *args, **kwargs)>)
(<class 'ta.momentum.KAMAIndicator'>, <Signature (self, close: pandas.core.series.Series, window: int = 10, pow1: int = 2, pow2: int = 30, fillna: bool = False)>)
(<class 'ta.momentum.PercentagePriceOscillator'>, <Signature (self, close: pandas.core.series.Series, window_slow: int = 26, window_fast: int = 12, window_sign: int = 9, fillna: bool = False)>)
(<class 'ta.momentum.PercentageVolumeOscillator'>, <Signature (self, volume: pandas.core.series.Series, window_slow: int = 26, window_fast: int = 12, window_sign: int = 9, fillna: bool = False)>)
(<class 'ta.momentum.ROCIndicator'>, <Signature (self, close: pandas.core.series.Series, window: int = 12, fillna: bool = False)>)
(<class 'ta.momentum.RSIIndicator'>, <Signatu

In [134]:
def inject_params(constructors):
    instances = dict()
    for constructor in constructors:
        init_signature = constructor[1]
        to_inject = dict()
        for param in init_signature.parameters.values():
            if param.name in ('self', 'args', 'kwargs'):
                continue
            if param.name in stock_data:
                to_inject[param.name] = stock_data[param.name]
            else:
                _type = type(param.default)
                value = None
                if _type is int:
                    value = random.randint(0, 100)
                if _type is float:
                    value = random.uniform(0, 100)
                if _type is bool:
                    value = False
                to_inject[param.name] = value
        instance = constructor[0]() if len(init_signature.parameters.values()) == 1 else constructor[0](**to_inject)
        instances['GENERATED_' + constructor[0].__name__] = instance
    return instances    
    
instances = inject_params(constructors)
print(instances)

{'GENERATED_AwesomeOscillatorIndicator': <ta.momentum.AwesomeOscillatorIndicator object at 0x000001A8922A35E0>, 'GENERATED_IndicatorMixin': <ta.utils.IndicatorMixin object at 0x000001A8922A16C0>, 'GENERATED_KAMAIndicator': <ta.momentum.KAMAIndicator object at 0x000001A8922E7670>, 'GENERATED_PercentagePriceOscillator': <ta.momentum.PercentagePriceOscillator object at 0x000001A8922E5AE0>, 'GENERATED_PercentageVolumeOscillator': <ta.momentum.PercentageVolumeOscillator object at 0x000001A8922E7EE0>, 'GENERATED_ROCIndicator': <ta.momentum.ROCIndicator object at 0x000001A8922E76D0>, 'GENERATED_RSIIndicator': <ta.momentum.RSIIndicator object at 0x000001A8923355A0>, 'GENERATED_StochRSIIndicator': <ta.momentum.StochRSIIndicator object at 0x000001A8EF180B20>, 'GENERATED_StochasticOscillator': <ta.momentum.StochasticOscillator object at 0x000001A891D686D0>, 'GENERATED_TSIIndicator': <ta.momentum.TSIIndicator object at 0x000001A8903BF3A0>, 'GENERATED_UltimateOscillator': <ta.momentum.UltimateOscil

In [135]:
def retrieve_features(instances):
    features = dict()
    for feature_name, instance in instances.items():
        method_names = [m for m in dir(instance) if m[0] != '_']
        for method_name in method_names:
            method = getattr(instance, method_name)
            feature_vals = method()
            if feature_vals.isna().sum() < 365:
                features[feature_name] = feature_vals
        if len(method_names) == 0:
            continue
    return features

features = retrieve_features(instances)
print(features)

{'GENERATED_AwesomeOscillatorIndicator': 100             NaN
101             NaN
102             NaN
103             NaN
104             NaN
           ...     
2278     560.619578
2279      89.917103
2280    -338.715557
2281    -744.142894
2282   -1068.856688
Name: ao, Length: 2183, dtype: float64, 'GENERATED_KAMAIndicator': 100              NaN
101              NaN
102              NaN
103              NaN
104              NaN
            ...     
2278    48487.029655
2279    48853.135897
2280    49096.053734
2281    49347.285641
2282    49447.186712
Name: kama, Length: 2183, dtype: float64, 'GENERATED_PercentagePriceOscillator': 100          NaN
101          NaN
102          NaN
103          NaN
104          NaN
          ...   
2278    2.502698
2279    1.805336
2280    1.175356
2281    0.588341
2282    0.179075
Name: PPO_sign_87_2, Length: 2183, dtype: float64, 'GENERATED_PercentageVolumeOscillator': 100          NaN
101          NaN
102          NaN
103          NaN
104          N